In [ ]:
from lusidtools.jupyter_tools import toggle_code

"""Amortisation RuleSet

Attributes
----------
Instruments
Transactions
Amortisation RuleSets
AmortisationRules
Recipe
Valuation
"""

toggle_code("Toggle Docstring")

# Amortisation Rule Sets

This example demonstrates how to create, update, and delete Amortisation Rule Sets and Rules, and how changes to the amortisation rules affect the amortisation valuation.

We will create bond instruments, add properties to a specific instrument, create an amortisation ruleset, set up a portfolio, add properties to the portfolio, and configure the amortisation rules.

Further we will upsert similar transactions to the respective bond instruments to demonstrate the effects of the amortisation rules through comparison. 

We will then create a simple recipe for valuation, including the valuation function.

Once Done, we will then demostrate how updating the amortisation rules impacts the valuation.

## Table of Contents:
- 1. [Imports](#1.-Imports)
- 2. [LUSID APIs](#2.-LUSID-APIs)
- 3. [Instrument Creation](#3.-Instrument-Creation)
- 4. [Amortisation RuleSet Creation](#4.-Amortisation-RuleSet-Creation)
- 5. [Portfolio Creation](#5.-Portfolio-Creation)
- 6. [Setting Amortisation Rule](#6.-Setting-Amortisation-Rule)
- 7. [Upsert Transactions](#7.-Upsert-Transactions)
- 8. [Recipe Creation](#8.-Recipe-Creation)
- 9. [Valuation](#9.-Valuation)
- 10. [Data Cleaning](#9.-Data-Cleaning)

# 1. Imports

In [ ]:
# Import necessary libraries
import lusid
import finbourne_access
import lusid.models as models
import openpyxl
import subprocess
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
    format_quotes_response,
)
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidjam import RefreshingToken
from fbnsdkutilities import ApiClientFactory
from finbourne_access.utilities import ApiClientFactory as AccessApiClientFactory
from finbourne_access import models as AccessModels
import os
import json
from IPython.display import Image
import pandas as pd
import pytz
from datetime import datetime
from lusidtools.lpt.lpt import to_date
pd.set_option("display.max_columns", None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

lusid_api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

api_client = lusid_api_factory.api_client

lusid_api_url = api_client.configuration.host
access_api_url = lusid_api_url[: lusid_api_url.rfind("/") + 1] + "access"
identity_api_url = lusid_api_url[: lusid_api_url.rfind("/") + 1] + "identity"

access_api_factory = finbourne_access.utilities.ApiClientFactory(
    token=api_client.configuration.access_token,
    access_url=access_api_url,
    app_name="LusidJupyterNotebook",
)
api_factory = ApiClientFactory(lusid, token=RefreshingToken())

# 2. LUSID APIs
Firstly, we initialize the LUSID APIs required for the notebook

In [ ]:
# Initiate the LUSID APIs required for the notebook

instruments_api = api_factory.build(lusid.api.InstrumentsApi)
amortisation_api = api_factory.build(lusid.api.AmortisationRuleSetsApi)
transaction_portfolio_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
portfolio_api = api_factory.build(lusid.api.PortfoliosApi)
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)
aggregation_api = api_factory.build(lusid.AggregationApi)
property_api = api_factory.build(lusid.PropertyDefinitionsApi)

# 3. Instrument Creation
We will create the relevant bond instruments

In [ ]:
#Setting Scope and Code to be used throughout the notebook

scope = "valuation-sample"
portfolio_code = "EQUITY_UK_1"

In [ ]:
#Defining function for bond instrument creation

def create_bond(currency,payment_frequency,roll_convention,day_count_convention,payment_calendars,reset_calendars,settle_days,reset_days,start_date,maturity_date,dom_ccy,
                principal,coupon_rate,bond_identifier,bond_name):

    flow_conventions = lusid.FlowConventions(
        currency=currency,
        payment_frequency=payment_frequency,
        roll_convention=roll_convention,
        day_count_convention=day_count_convention,
        payment_calendars=payment_calendars,
        reset_calendars=reset_calendars,
        settle_days=settle_days,
        reset_days=reset_days,
    )

    bond = lusid.Bond(
        start_date=start_date,
        maturity_date=maturity_date,
        dom_ccy=dom_ccy,
        principal=principal,
        coupon_rate=coupon_rate,
        flow_conventions=flow_conventions,
        identifiers={},
        instrument_type="Bond",
        calculation_type="Standard",
    )

    # define the instrument to be upserted
    bond_definition = lusid.InstrumentDefinition(
        name=bond_name,
        identifiers={"ClientInternal": lusid.InstrumentIdValue(bond_identifier)},
        definition=bond,
    )

    # upsert the instrument
    upsert_request = {bond_identifier: bond_definition}
    upsert_response = instruments_api.upsert_instruments(request_body=upsert_request)
    bond_luid = upsert_response.values[bond_identifier].lusid_instrument_id
    return(bond_luid)

In [ ]:
#Creating first Bond Instrument

currency = "GBP"
payment_frequency = "1M"
roll_convention = "none"
day_count_convention = "Actual365"
payment_calendars = []
reset_calendars = []
settle_days = 0
reset_days = 0
start_date = datetime(2024, 1, 15, 0, 0, tzinfo=pytz.utc).isoformat()
maturity_date = datetime(2024, 8, 15, 0, 0, tzinfo=pytz.utc).isoformat()
dom_ccy = "GBP"
principal = 10487
coupon_rate = 0.07
bond_identifier = "Example_Bond_A"
bond_name = "ExampleBond_A"

luid = create_bond(currency, payment_frequency, roll_convention, day_count_convention, payment_calendars, reset_calendars, settle_days, reset_days, start_date, maturity_date, dom_ccy, 
                   principal, coupon_rate, bond_identifier, bond_name)

luid

In [ ]:
#Creating second Bond Instrument

currency = "GBP"
payment_frequency = "1M"
roll_convention = "none"
day_count_convention = "Actual365"
payment_calendars = []
reset_calendars = []
settle_days = 0
reset_days = 0
start_date = datetime(2024, 1, 15, 0, 0, tzinfo=pytz.utc).isoformat()
maturity_date = datetime(2024, 8, 15, 0, 0, tzinfo=pytz.utc).isoformat()
dom_ccy = "GBP"
principal = 10487
coupon_rate = 0.07
bond_identifier = "Example_Bond_B"
bond_name = "ExampleBond_B"

luid_1=create_bond(currency, payment_frequency, roll_convention, day_count_convention, payment_calendars, reset_calendars, settle_days, reset_days, start_date, maturity_date, dom_ccy,
    principal, coupon_rate, bond_identifier, bond_name)

luid_1

# 3.1 Instrument Property
We will create and upsert instrument property for the first instrument so that it can be used when defining amortisation rules

We will do so only for one instrument so that impact of amortisation rules on instruments can be compared based on instrument properties

In [ ]:
#Creating Instrument Property
try:
    instrument_prop = property_api.create_property_definition(
        create_property_definition_request=lusid.CreatePropertyDefinitionRequest(
            domain = "Instrument",
            scope = scope,
            code = portfolio_code,
            data_type_id= lusid.ResourceId(
                scope = "system",
                code = "string",
            ),
            life_time= "Perpetual",
            constraint_style = "Property",
            display_name = "Bond_Prop_1",
            property_description = "Example Bond Property",
        )
        
    )
    print("Instrument Property Created")
except lusid.ApiException as e:
    print(json.loads(e.body)["title"])

In [ ]:
#Upserting property to first Bond Instrument
try:
    upsert_instrument_prop = instruments_api.upsert_instruments_properties(
        upsert_instrument_property_request=[
            lusid.UpsertInstrumentPropertyRequest(
                identifier_type="LusidInstrumentId",
                identifier=luid,
                properties=[
                    lusid.ModelProperty(
                        key="Instrument/valuation-sample/EQUITY_UK_1",
                        value=lusid.PropertyValue(
                            label_value='alpha1'
                        )
                    )
                ]
            )
        ]
    )
    print("Instrument Property Upserted")
except lusid.ApiException as e:
    print(json.loads(e.body)["title"])

# 4. Amortisation RuleSet Creation
We will create the relevant Amortisation Rule Set

In [ ]:
#Creating Amortisation Ruleset
try:
    create_amt = amortisation_api.create_amortisation_rule_set(
        scope= scope,
        create_amortisation_rule_set_request=models.CreateAmortisationRuleSetRequest(
            code=portfolio_code,
            display_name='Example_Amortisation_Rule',
            description='This is an example run'
        )
    )
    print("Amortisation RuleSet Created")
except lusid.ApiException as e:
    print(json.loads(e.body)["title"])

# 5. Portfolio Creation
We proceed by creating a basic transaction portfolio

In [ ]:
# Creating a Transaction Portfolio

try:
    create_portfolio_response = transaction_portfolio_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=models.CreateTransactionPortfolioRequest(
            display_name="RuleSet_Portfolio",
            code=portfolio_code,
            base_currency="GBP",
            created=datetime(2024, 1, 1, 0, 0, tzinfo=pytz.utc).isoformat(),
            sub_holding_keys=[],
            amortisation_method='NoAmortisation',
            amortisation_rule_set_id={
                "scope": scope,
                "code": portfolio_code
            }

        ),
    )

    print("Portfolio Created")

except lusid.ApiException as e:
    print(json.loads(e.body)["title"])


# 5.1 Portfolio Property
We will create and upsert portfolio property for the relevant portfolio

In [ ]:
#Creating Portfolio Property
try:
    property_creation = property_api.create_property_definition(
        create_property_definition_request=lusid.CreatePropertyDefinitionRequest(
            domain = "Portfolio",
            scope = scope,
            code = portfolio_code,
            data_type_id= lusid.ResourceId(
                scope = "system",
                code = "string",
            ),
            life_time= "Perpetual",
            constraint_style = "Property",
            display_name = "Portfolio_property",
            property_description = "Example amortisation rule Property",
        )
        
    )
    print('Portfolio Property Created')
except lusid.ApiException as e:
    print(json.loads(e.body)["title"])

In [ ]:
#Upserting property to Portfolio
try:
    upsert_port= portfolio_api.upsert_portfolio_properties(
        scope= scope,
        code= portfolio_code,
        request_body={
            "Portfolio/valuation-sample/EQUITY_UK_1": {
                
                "key": "Portfolio/valuation-sample/EQUITY_UK_1",
                "value": {
                    "labelValue": "alpha1"
                },
        },
        }
        
    )  
    print("Portfolio Property Upserted")
except lusid.ApiException as e:
    print(json.loads(e.body)["title"])

# 6. Setting Amortisation Rule
We will upsert rules to the Amortisation Rule Set

In [ ]:
#Set Amortisation Rules

set_amt_rule = amortisation_api.set_amortisation_rules(
    scope = scope,
    code = portfolio_code,
    set_amortisation_rules_request=models.SetAmortisationRulesRequest(
        rules_interval=models.RulesInterval(
            effective_range={
                "fromDate": "0001-01-01T00:00:00.0000000+00:00",
            },
            rules=[]
        )
    )
)

# 6.1 List Amortisation RuleSets
Using this endpoint we can see the list of Amortisation RuleSets and associates Rules

In [ ]:
#List RuleSets

list_amt= amortisation_api.list_amortisation_rule_sets()
list_amt

# 7. Upsert Transactions
We will upsert similar transactions for the created instruments to observe the impacts of the amortisation rules

In [ ]:
transactions = pd.read_csv("Transaction_Data.csv")
transactions

In [ ]:
#Upserting Transactions
transaction_request = [
    lusid.TransactionRequest(
        transaction_id=row["txn_id"],
        type=row["type"],
        instrument_identifiers={
            "Instrument/default/ClientInternal": row["client_id"]
        },
        transaction_date=to_date(row["trade_date"]).isoformat(),
        settlement_date=to_date(row["settlement_date"]).isoformat(),
        units=row["quantity"],
        transaction_price=lusid.TransactionPrice(price=row["price"], type="Price"),
        total_consideration=lusid.CurrencyAndAmount(
            amount=row["total_consideration"], currency=row["currency"]
        ),
    )
    for index, row in transactions.iterrows()
]

response = transaction_portfolio_api.upsert_transactions(
    scope=scope, code=portfolio_code, transaction_request=transaction_request
)

print(f"Transactions succesfully updated at time: {response.version.as_at_date}")

# 8. Recipe Creation

Following the initial setup, we can see to configuring how LUSID will conduct valuation. This introduces the concept of recipes, which are a set of steps we specify to the valuation engine relating to market data and model specification.


In [ ]:
# Creating Recipe to perform a valuation
try:
    configuration_recipe = models.ConfigurationRecipe(
        scope=scope,
        code=portfolio_code,
        market=models.MarketContext(
            market_rules=[
                # define how to resolve the quotes
                models.MarketDataKeyRule(
                    key="Quote.ClientInternal.*",
                    supplier="Lusid",
                    data_scope=scope,
                    quote_type="Price",
                    field= "bid",
                ),
            ],
            options=models.MarketOptions(
                default_supplier="Lusid",
                default_instrument_code_type="ClientInternal",
                default_scope=scope,
            ),
        ),
        pricing=models.PricingContext(
            options={"AllowPartiallySuccessfulEvaluation": True},
        ),
    )
    
    upsert_configuration_recipe_response = configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=models.UpsertRecipeRequest(
            configuration_recipe=configuration_recipe
        )
    )
    print("Recipe Created Successfully")
except lusid.ApiException as e:
    print(json.loads(e.body)["title"])

# 9. Valuation

In [ ]:
#Defining function for Valuation

def generate_valuation_request(valuation_effectiveAt):

    # Create the valuation request
    valuation_request = models.ValuationRequest(
        recipe_id=models.ResourceId(
            scope=scope, code=portfolio_code
        ),
        metrics=[
            models.AggregateSpec("Instrument/default/Name", "Value"),
            models.AggregateSpec("Valuation/PvInReportCcy", "Proportion"),
            models.AggregateSpec("Valuation/PvInReportCcy", "Sum"),
            models.AggregateSpec("Holding/default/Units", "Sum"),
            models.AggregateSpec("Holding/AmortisedCost", "Value"),
        ],
        group_by=["Instrument/default/Name"],
        portfolio_entity_ids=[
            models.PortfolioEntityId(scope=scope, code=portfolio_code)
        ],
        valuation_schedule=models.ValuationSchedule(
            effective_at=valuation_effectiveAt.isoformat()
        ),
    )

    return valuation_request

In [ ]:
#Getting Valuation

aggregation = aggregation_api.get_valuation(
    valuation_request=generate_valuation_request(datetime(year=2024, month=4, day=25, tzinfo=pytz.UTC)
    )
)

pd.DataFrame(aggregation.data)

We can see that the Amortised Cost is calculated based on No Amortisation method as set when creating the portfolio. No Amortisation Rules were defined at this point 

# 9.1 Defining and Setting Amortisation Rules

We can change/update Amortisation Rules based on the filters we provide. Following are a few examples of how we can use filters to set amortisation rules, along with imSetting Amortisation Rule filter to True eq True. This means that the amortisation method set in the rule defined will take precedence for all transactions in the relevant portfolio

In [ ]:
#Updating Amortisation Rule 

set_amt_rule = amortisation_api.set_amortisation_rules(
    scope = scope,
    code = portfolio_code,
    set_amortisation_rules_request=models.SetAmortisationRulesRequest(
        rules_interval=models.RulesInterval(
            effective_range={
                "fromDate": "0001-01-01T00:00:00.0000000+00:00",
            },
            rules=[
                {
                    "name" : "Amortisation_Rule_1",
                    "description" : "Rule setting filter to True equals True",
                    "filter" : "True eq True",
                    "amortisationMethod" : "StraightLine",
                },
            ]
        )
    )
)

In [ ]:
#Getting Valuation

aggregation = aggregation_api.get_valuation(
    valuation_request=generate_valuation_request(datetime(year=2024, month=4, day=25, tzinfo=pytz.UTC)
    )
)

pd.DataFrame(aggregation.data)

We can see that the Amortised Cost is calculate based on StraightLine method as updated in the rule.

Setting Amortisation Rule filter based on instrument property. This means that the amortisation method set in the rule defined will take precedence over the method set on portfolio creation, for the instrument with the relevant property that is used in the filter

In [ ]:
#Updating Amortisation Rule

set_amt_rule = amortisation_api.set_amortisation_rules(
    scope = scope,
    code = portfolio_code,
    set_amortisation_rules_request=models.SetAmortisationRulesRequest(
        rules_interval=models.RulesInterval(
            effective_range={
                "fromDate": "0001-01-01T00:00:00.0000000+00:00",
            },
            rules=[
                {
                    "name" : "Amortisation_Rule_1",
                    "description" : "Rule Description",
                    "filter" : "properties[Instrument/valuation-sample/EQUITY_UK_1] eq 'alpha1'",
                    "amortisationMethod" : "StraightLine",
                },
            ]
        )
    )
)

In [ ]:
#Getting Valuation

aggregation = aggregation_api.get_valuation(
    valuation_request=generate_valuation_request(datetime(year=2024, month=4, day=25, tzinfo=pytz.UTC)
    )
)

pd.DataFrame(aggregation.data)

We can see that the Amortised Cost is calculate based on StraightLine method as updated in the rule for ExampleBond_A as it has the relevant property whereas ExampleBond_B has amortised cost based on NoAmortisation method

Now, setting Amortisation Rule filter based on Portfolio property. This means that the amortisation method set in the rule defined will take precedence over the method set on portfolio creation, for the instruments belonging to that portfolio.

In [ ]:
#Set Amortisation Rules

set_amt_rule = amortisation_api.set_amortisation_rules(
    scope = scope,
    code = portfolio_code,
    set_amortisation_rules_request=models.SetAmortisationRulesRequest(
        rules_interval=models.RulesInterval(
            effective_range={
                "fromDate": "0001-01-01T00:00:00.0000000+00:00",
            },
            rules=[
                {
                    "name" : "Amortisation_Rule_2",
                    "description" : "Rule Description",
                    "filter" : "properties[Portfolio/valuation-sample/EQUITY_UK_1] eq 'alpha1'",
                    "amortisationMethod" : "EffectiveYield",
                },
                
            ]
        )
    )
)

In [ ]:
#Getting Valuation

aggregation = aggregation_api.get_valuation(
    valuation_request=generate_valuation_request(datetime(year=2024, month=4, day=25, tzinfo=pytz.UTC)
    )
)

pd.DataFrame(aggregation.data)

We can see that the Amortised Cost is calculated based on EffectiveYield method as updated in the rule.

Now, we will show that when multiple rules exist, the first one that matches is implemented.

In [ ]:
#Set Amortisation Rules

set_amt_rule = amortisation_api.set_amortisation_rules(
    scope = scope,
    code = portfolio_code,
    set_amortisation_rules_request=models.SetAmortisationRulesRequest(
        rules_interval=models.RulesInterval(
            effective_range={
                "fromDate": "0001-01-01T00:00:00.0000000+00:00",
            },
            rules=[
                {
                    "name" : "Amortisation_Rule_1",
                    "description" : "Rule Description",
                    "filter" : "properties[Instrument/valuation-sample/EQUITY_UK_1] eq 'alpha1'",
                    "amortisationMethod" : "StraightLine",
                },
                {
                    "name" : "Amortisation_Rule_2",
                    "description" : "Rule Description",
                    "filter" : "properties[Portfolio/valuation-sample/EQUITY_UK_1] eq 'alpha1'",
                    "amortisationMethod" : "EffectiveYield",
                },
                
            ]
        )
    )
)

In [ ]:
#Getting Valuation

aggregation = aggregation_api.get_valuation(
    valuation_request=generate_valuation_request(datetime(year=2024, month=4, day=25, tzinfo=pytz.UTC)
    )
)

pd.DataFrame(aggregation.data)

We can see that the Amortised Cost is calculated based on StraightLine method for ExampleBond_A as it matches the first rule where as EffectiveYield method is used for ExampleBond_B as it matches the second rule in the list.

# 10. Data Cleaning

In [ ]:

#Delete Instruments
def DeleteInstruments(luid): 
    try:
        delete_instrument = instruments_api.delete_instrument(
            identifier_type="LusidInstrumentId", identifier= luid
        )
    
        print(delete_instrument)
    
    except lusid.ApiException as e:
        print(json.loads(e.body)["title"])


In [ ]:
'''
#Deleting Instruments

DeleteInstruments(luid)
DeleteInstruments(luid_1)

In [ ]:
'''
#Delete Rule

delete_amt= amortisation_api.delete_amortisation_ruleset(
    scope=scope,
    code=portfolio_code
)

delete_amt

In [ ]:
'''
#Delete Portfolio
try:
    delete_portfolio = portfolio_api.delete_portfolio(scope, portfolio_code)
        
    print(delete_portfolio)

except lusid.ApiException as e:
    print(json.loads(e.body)["title"])


In [ ]:
'''
#Delete Recipe
try:
    delete_recipe = configuration_recipe_api.delete_configuration_recipe(
        scope=scope,
        code=portfolio_code,
    )

    print(delete_recipe)

except lusid.ApiException as e:
    print(json.loads(e.body)["title"])

In [ ]:
'''
#Delete Portfolio Properties
try:
    delete_portfolio_prop = property_api.delete_property_definition(
        domain= 'Portfolio',
        scope= scope,
        code= portfolio_code,
    )
    print(delete_portfolio_prop)
except lusid.ApiException as e:
    print(json.loads(e.body)["title"])

In [ ]:
'''
#Delete Instrument Properties
try:
    delete_instrument_prop = property_api.delete_property_definition(
        domain= 'Instrument',
        scope= scope,
        code= portfolio_code,
    )
    print(delete_instrument_prop)
except lusid.ApiException as e:
    print(json.loads(e.body)["title"])